<a href="https://colab.research.google.com/github/bemarina/text-clf-deep-learning/blob/main/pretrained-word-embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Example with pretrained word embeddings (Keras documentation)

###Import modules

In [1]:
import os

# Only the TensorFlow backend supports string inputs.
os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import numpy as np
import tensorflow.data as tf_data
import keras
from keras import layers

###Download and explore the data

In [2]:
#Download the Newsgroup20 data
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17329808/17329808 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])


Number of directories: 20
Directory names: ['talk.politics.guns', 'sci.med', 'talk.politics.mideast', 'sci.electronics', 'comp.sys.ibm.pc.hardware', 'alt.atheism', 'rec.autos', 'misc.forsale', 'sci.space', 'talk.politics.misc', 'comp.os.ms-windows.misc', 'sci.crypt', 'rec.sport.baseball', 'talk.religion.misc', 'soc.religion.christian', 'comp.windows.x', 'comp.sys.mac.hardware', 'rec.sport.hockey', 'comp.graphics', 'rec.motorcycles']
Number of files in comp.graphics: 1000
Some example filenames: ['39640', '38314', '38962', '38947', '38866']


In [4]:
data_dir

PosixPath('/root/.keras/datasets/20_newsgroup')

###Split data into training and validation sets

In [6]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

In [10]:
len(class_names)
# 20 classes
# almost 1000 files per class, so we can assume it's a balanced clf problem

20

In [11]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]


In [15]:
print(len(samples))
samples[0]
print(len(train_samples))
print(len(val_samples))

19997
15998
3999


###Vocabulary index

In [16]:
# Use only 20,000 words and truncate/pad as necessary
vectorizer = layers.TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf_data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)